## Shop Store Example

![image](images/diagram-01.jpeg)

#### ¿Por qué utilizar una sola tabla en lugar de varias?
Porque nuestros datos forman parte del mismo conjunto de datos. El concepto de tabla en DynamoDB es comparable al concepto de base de datos en motores como Postgres o MySQL. Si los datos están relacionados y necesitamos consultarlos juntos, van a la misma tabla de DynamoDB.
Si estuviéramos construyendo microservicios, cada microservicio tendría su propia Tabla, porque tiene sus propios datos.

#### ¿Qué son los índices secundarios locales y los índices secundarios globales en DynamoDB?
Son estructuras de datos que contienen un subconjunto de atributos de una tabla y una clave principal diferente. Los índices secundarios locales tienen la misma clave de partición y una clave de ordenación diferente, mientras que los índices secundarios globales tienen una clave de partición y una clave de ordenación diferentes. Usted define los atributos que desea proyectar en el índice, y DynamoDB copia estos atributos en el índice, junto con los atributos de clave primaria de la tabla base. A continuación, puede consultar o escanear el índice del mismo modo que consultaría o escanearía una tabla.
Las consultas contra la clave primaria son realmente rápidas y baratas, y las consultas que no son contra la clave primaria son realmente lentas y caras. Los índices nos proporcionan una clave primaria diferente para los mismos atributos, por lo que podemos consultar los mismos datos de diferentes maneras.


[Link to Access Patterns](https://docs.google.com/spreadsheets/d/1TU0IrwYBBC6xzqyx_jJkOjZaGuq2Cd2i4xngqFGsS6Y/edit?usp=sharing)

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import base64
from spdynamodb import DynamoTable
from datetime import datetime
import json
import time
from decimal import Decimal

In [2]:
dt = DynamoTable()
try:
    dt.select_table('ShopSampleTable')
    print(dt)
except:
    dt.create_table(
        table_name='ShopSampleTable',
        partition_key='PK',
        partition_key_type='S',
        sort_key="SK",
        sort_key_type="S"
)

- Table name: ShopSampleTable            
- Table arn: arn:aws:dynamodb:us-east-1:637423169504:table/ShopSampleTable            
- Table creation: 2024-07-22 14:38:40            
- [{'AttributeName': 'PK', 'KeyType': 'HASH'}, {'AttributeName': 'SK', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'GSI1-PK', 'AttributeType': 'S'}, {'AttributeName': 'GSI1-SK', 'AttributeType': 'S'}, {'AttributeName': 'GSI2-PK', 'AttributeType': 'S'}, {'AttributeName': 'GSI2-SK', 'AttributeType': 'S'}, {'AttributeName': 'PK', 'AttributeType': 'S'}, {'AttributeName': 'SK', 'AttributeType': 'S'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


In [ ]:
dt.create_global_secondary_index(
    att_name="GSI1-PK",
    att_type="S",
    sort_index="GSI1-SK",
    sort_type="S",
    index_name="GSI1",
    read_capacity=5,
    write_capacity=5
)

In [ ]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    time.sleep(30)
print("Global secondary index created.")

In [ ]:
dt.create_global_secondary_index(
    att_name="GSI2-PK",
    att_type="S",
    sort_index="GSI2-SK",
    sort_type="S",
    index_name="GSI2",
    read_capacity=5,
    write_capacity=5
)

In [ ]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    time.sleep(30)
print("Global secondary index created.")

### Add Customers

In [ ]:
dt.add_item({
    "PK": "c#20656",
    "SK": "c#20656",
    "EntityType": "Client",
    "Email": "joe@example.com",
    "FirstName": "Joe",
    "LastName": "Smith",
    "Addres": {
        "Street": "123 Main St",
        "City": "Anytown",
        "State": "CA",
        "Zip": "12345"
    },
    "Date": "2023-12-05T12:33:10"
})
dt.add_item({
    "PK": "c#81294",
    "SK": "c#81294",
    "EntityType": "Client",
    "Email": "jane.doe@email.com",
    "FirstName": "Jane",
    "LastName": "Doe",
    "Addres": {  
        "Street": "45 Elm Street",
        "City": "Springfield",
        "State": "IL",
        "Zip": "62704"
    },
    "Date": "2024-02-18T09:15:42" 
})
dt.add_item({
    "PK": "c#39712",
    "SK": "c#39712",
    "EntityType": "Client",
    "Email": "michael.johnson@domain.net",
    "FirstName": "Michael",
    "LastName": "Johnson",
    "Addres": {  
        "Street": "987 Oak Avenue",
        "City": "Austin",
        "State": "TX",
        "Zip": "78701"
    },
    "Date": "2023-08-22T16:08:33" 
})
dt.add_item({
    "PK": "c#55038",
    "SK": "c#55038",
    "EntityType": "Client",
    "Email": "emily.davis@company.org",
    "FirstName": "Emily",
    "LastName": "Davis",
    "Addres": {  
        "Street": "65 Pine Lane",
        "City": "New York",
        "State": "NY",
        "Zip": "10001"
    },
    "Date": "2023-05-03T11:29:57" 
})

### Add Products

In [ ]:
dt.add_item({
    "PK": "p#1244",
    "SK": "p#1244",
    "EntityType": "Product",
    "Detail": {
        "Name": "War and Peace",
        "Description": "Novel by Leon Tolstoy"
    },
    "Stock": 39,
    "Price": 12.99,
    "Date": "2023-05-03T11:29:57" 
})
dt.add_item({
    "PK": "p#6745",
    "SK": "p#6745",
    "EntityType": "Product",
    "Detail": {
        "Name": "Building Block Set - 500 Pieces",
        "Description": "Colorful plastic building blocks for creative construction",
        "Category": "Toys",
        "AgeRange": "3+",
        "Brand": "BlockMasters"
    },
    "Stock": 85,
    "Price": 24.99,
    "Date": "2023-11-18T10:35:08" 
})
dt.add_item({
    "PK": "p#3398",
    "SK": "p#3398",
    "EntityType": "Product",
    "Detail": {
        "Name": "Stainless Steel Kitchen Knife Set - 5 Piece",
        "Description": "High-quality knife set for professional and home chefs",
        "Category": "Kitchen",
        "Material": "Stainless Steel",
        "Brand": "CutleryPro"
    },
    "Stock": 32,
    "Price": 49.95,
    "Date": "2023-09-25T14:10:22" 
})
dt.add_item({
    "PK": "p#7129",
    "SK": "p#7129",
    "EntityType": "Product",
    "Detail": {
        "Name": "Men's Denim Jeans - Slim Fit",
        "Description": "Classic slim-fit jeans in dark wash denim",
        "Category": "Clothing",
        "SubCategory": "Men's",
        "Size": "32W 32L",
        "Color": "Dark Blue",
        "Brand": "UrbanWear"
    },
    "Stock": 18,
    "Price": 59.00,
    "Date": "2023-12-03T08:55:17" 
})


### Add Orders

In [ ]:
dt.add_item({
    "PK": "o#39488",
    "SK": "c#20656",
    "EntityType": "Order",
    "OrderDate": "2023-07-15T10:35:28",
    "CustomerName": "John Doe",
    "ShippingAddress": {
        "Street": "123 Main St",
        "City": "Anytown",
        "State": "CA",
        "ZIP": "12345"
    },
    "Items": [
        {"ProductSK": "p#2891", "Quantity": 2, "Price": 14.99},
        {"ProductSK": "p#5603", "Quantity": 1, "Price": 29.95}
    ],
    "OrderStatus": "Pending"
})
dt.add_item({
    "PK": "o#81255",
    "SK": "c#98765",
    "EntityType": "Order",
    "OrderDate": "2023-11-22T18:12:05",
    "CustomerName": "Jane Smith",
    "ShippingAddress": {
        "Street": "456 Elm St",
        "City": "Otherville",
        "State": "NY",
        "ZIP": "54321"
    },
    "Items": [
        {"ProductSK": "p#6745", "Quantity": 3, "Price": 9.50}
    ],
    "OrderStatus": "Shipped"
})
dt.add_item({
    "PK": "o#10032",
    "SK": "c#45123",
    "EntityType": "Order",
    "OrderDate": "2024-01-08T09:00:43",
    "CustomerName": "Mike Johnson",
    "ShippingAddress": {
        "Street": "789 Oak St",
        "City": "Someplace",
        "State": "FL",
        "ZIP": "98765"
    },
    "Items": [
        {"ProductSK": "p#9822", "Quantity": 1, "Price": 19.99},
        {"ProductSK": "p#3398", "Quantity": 1, "Price": 35.00}
    ],
    "OrderStatus": "Delivered"
})

### Add OderItems

In [ ]:
p#7129

In [ ]:
dt.add_item({
    "PK": "o#39488",
    "SK": "p#6745",
    "EntityType": "OrderItem",
    "GSI1-PK": "p#6745",
    "GSI1-SK": "2023-11-06T18:07:00",
    "GSI2-PK": "c#20656",
    "GSI2-SK": "p#2023-11-06T18:07:00",
    "Price": 20.99,
    "Quantity": 4
})
dt.add_item({
    "PK": "o#39488",
    "SK": "p#7129",
    "EntityType": "OrderItem",
    "GSI1-PK": "p#7129",
    "GSI1-SK": "2024-06-11T09:02:14",
    "GSI2-PK": "c#20656",
    "GSI2-SK": "p#2024-06-11T09:02:14",
    "Price": 20.99,
    "Quantity": 4
})
dt.add_item({
    "PK": "o#31655",
    "SK": "p#7129",
    "EntityType": "OrderItem",
    "GSI1-PK": "p#7129",
    "GSI1-SK": "2022-12-05T12:02:14",
    "GSI2-PK": "c#20656",
    "GSI2-SK": "p#2022-12-05T12:02:14",
    "Price": 20.99,
    "Quantity": 4
})
dt.add_item({
    "PK": "o#36885",
    "SK": "p#7129",
    "EntityType": "OrderItem",
    "GSI1-PK": "p#7129",
    "GSI1-SK": "2021-02-10T03:02:14",
    "GSI2-PK": "c#20656",
    "GSI2-SK": "p#2021-02-10T03:02:14",
    "Price": 20.99,
    "Quantity": 4
})

### Add Invoice

In [6]:
dt.add_item({
    "PK": "o#39488",
    "SK": "i#15889",
    "EntityType": "Invoice",
    "GSI1-PK": "i#15889",
    "GSI1-SK": "i#15889",
    "GSI2-PK": "c#20656",
    "GSI2-SK": "i#2024-05-15T08:10:14",
    "Amount": 125.55,
    "Date": "2024-05-15T08:10:14"
})
dt.add_item({
    "PK": "o#39488",
    "SK": "i#85466",
    "EntityType": "Invoice",
    "GSI1-PK": "i#85466",
    "GSI1-SK": "i#85466",
    "GSI2-PK": "c#20656",
    "GSI2-SK": "i#2022-02-18T08:10:14",
    "Amount": 203.99,
    "Date": "2022-02-18T08:10:14"
})
dt.add_item({
    "PK": "o#39488",
    "SK": "i#66584",
    "EntityType": "Invoice",
    "GSI1-PK": "i#66584",
    "GSI1-SK": "i#66584",
    "GSI2-PK": "c#20656",
    "GSI2-SK": "i#2023-09-22T08:10:14",
    "Amount": 70.99,
    "Date": "2023-09-22T08:10:14"
})

### 1. Get customer for a given customerId

In [ ]:
dt.get_item(
    pk_value="c#20656",
    sk_value="c#20656"
)

### 2. Get product for a given productId

In [ ]:
dt.get_item(
    pk_value="p#1244",
    sk_value="p#1244"
)

### 3. Get order for a given orderId

In [ ]:
dt.query(
    pk_value="o#39488",
    to_pandas=True
)

### 4. Get all products for a given orderId

In [ ]:
dt.query(
    pk_value="o#39488",
    sk_value="p#*",
    to_pandas=True
)

### 5. Get invoice for a given orderId

In [ ]:
dt.query(
    pk_value="o#39488",
    sk_value="i#*",
    to_pandas=True
)

### 6. Get all orders for a given productId for a given date range

In [13]:
dt.query(
    pk_value="p#7129",
    sk_value="2021-02-10T03:02:14_2024-12-10T12:10:14",
    index_name="GSI1",
    to_pandas=True
)

,GSI2-SK,Price,SK,GSI1-SK,GSI1-PK,Quantity,GSI2-PK,PK,EntityType
0,p#2021-02-10T03:02:14,20.99,p#7129,2021-02-10T03:02:14,p#7129,4,c#20656,o#36885,OrderItem
1,p#2022-12-05T12:02:14,20.99,p#7129,2022-12-05T12:02:14,p#7129,4,c#20656,o#31655,OrderItem
2,p#2024-06-11T09:02:14,20.99,p#7129,2024-06-11T09:02:14,p#7129,4,c#20656,o#39488,OrderItem


### 7. Get invoice for a given invoiceId

In [ ]:
dt.query(
    pk_value="i#15889",
    sk_value="i#15889",
    index_name="GSI1"
)

### 8. Get all payments for a given invoiceId

In [ ]:
# ToDo

### 9. Get all invoices for a given customerId for a given date range

In [3]:
dt.query(
    pk_value="c#20656",
    sk_value="i#2023-04-01_i#2024-12-30",
    index_name="GSI2"
)

[{'Date': '2023-09-22T08:10:14',
  'GSI2-SK': 'i#2023-09-22T08:10:14',
  'GSI1-SK': 'i#66584',
  'SK': 'i#66584',
  'GSI1-PK': 'i#66584',
  'GSI2-PK': 'c#20656',
  'Amount': 70.99,
  'PK': 'o#39488',
  'EntityType': 'Invoice'},
 {'Date': '2024-05-15T08:10:14',
  'GSI2-SK': 'i#2024-05-15T08:10:14',
  'GSI1-SK': 'i#15889',
  'SK': 'i#15889',
  'GSI1-PK': 'i#15889',
  'GSI2-PK': 'c#20656',
  'Amount': 125.55,
  'PK': 'o#39488',
  'EntityType': 'Invoice'}]

### 10. Get all products ordered by a given customerId for a given date range

In [18]:
dt.query(
    pk_value="c#20656",
    sk_value="p#2021-04-01_p#2024-12-30",
    index_name="GSI2",
    consumed_capacity=True
)

Consumed Capacity: 0.5


[{'GSI2-SK': 'p#2022-12-05T12:02:14',
  'GSI1-SK': '2022-12-05T12:02:14',
  'Price': 20.99,
  'SK': 'p#7129',
  'GSI1-PK': 'p#7129',
  'GSI2-PK': 'c#20656',
  'Quantity': 4,
  'PK': 'o#31655',
  'EntityType': 'OrderItem'},
 {'GSI2-SK': 'p#2023-11-06T18:07:00',
  'GSI1-SK': '2023-11-06T18:07:00',
  'Price': 20.99,
  'SK': 'p#6745',
  'GSI1-PK': 'p#6745',
  'GSI2-PK': 'c#20656',
  'Quantity': 4,
  'PK': 'o#39488',
  'EntityType': 'OrderItem'},
 {'GSI2-SK': 'p#2024-06-11T09:02:14',
  'GSI1-SK': '2024-06-11T09:02:14',
  'Price': 20.99,
  'SK': 'p#7129',
  'GSI1-PK': 'p#7129',
  'GSI2-PK': 'c#20656',
  'Quantity': 4,
  'PK': 'o#39488',
  'EntityType': 'OrderItem'}]

### Prácticas recomendadas para Amazon DynamoDB

#### Excelencia operativa
* Elija la consistencia de lectura correcta: Las lecturas de DynamoDB son finalmente consistentes por defecto. También puede realizar lecturas fuertemente consistentes, que cuestan el doble.

* Utilice transacciones cuando sea necesario: Las operaciones son atómicas, pero si necesita realizar más de una operación de forma atómica, puede utilizar una transacción. El coste es el doble de las operaciones normales.

* Monitorizar y optimizar: CloudWatch ofrece información muy útil sobre cómo se utiliza DynamoDB. Utilice esta información para optimizar su tabla.

#### Seguridad
* Utilice permisos IAM y privilegios mínimos: Debe conceder permisos para DynamoDB explícitamente, utilizando IAM. Puede otorgar permisos a su rol de IAM solo en una tabla, solo para algunas operaciones, e incluso puede hacerlo por elemento o por atributo. Conceda los permisos mínimos necesarios, no más.

#### Fiabilidad
* Añade una cola SQS para limitar las escrituras: En el modo aprovisionado, si excedes las unidades de capacidad de escritura disponibles, tu operación fallará. Su backend puede volver a intentarlo, pero esto aumenta el tiempo de respuesta y añade aún más carga a la tabla de DynamoDB. En su lugar, considere hacer la escritura asíncrona enviando todas las escrituras a una cola SQS y haciendo que un proceso consuma desde la cola SQS a un ritmo controlado. Esto es especialmente importante cuando se utilizan funciones Lambda, ya que tienden a superar la escala del modo aprovisionado de DynamoDB durante grandes picos.

* Realice copias de seguridad de los datos: Puede configurar copias de seguridad programadas o hacerlas bajo demanda. En cualquier caso, haga copias de seguridad.

* Considere una tabla global: DynamoDB tiene una función llamada Tabla Global, que es básicamente una entidad única y global respaldada por tablas regulares en diferentes regiones. Es la mejor opción para cualquier tipo de configuración multirregión, incluida la recuperación ante desastres.

#### Rendimiento eficaz
* Diseñe las claves de partición con cuidado: DynamoDB utiliza varios nodos entre bastidores, y la clave de partición es lo que determina qué nodo almacena qué elemento. Si elige una clave de partición incorrecta, la mayoría de las solicitudes se dirigirán al mismo nodo, lo que afectará al rendimiento. Elija una clave de partición que esté distribuida uniformemente, como un ID aleatorio. Aquí hay una gran lectura sobre el tema.

* Consulta siempre sobre índices: Cuando consulta un índice, DynamoDB solo lee los elementos que coinciden con la consulta y solo le cobra por ello. Cuando consulta un atributo no indexado, DynamoDB explora toda la tabla y le cobra por leer cada elemento (después los filtra).

* Utilice Consulta, no Exploración: La exploración lee toda la tabla, mientras que la consulta utiliza un índice. La exploración sólo debe utilizarse para atributos no indexados o para leer todos los elementos. No los mezcle.

* Utilice el almacenamiento en caché: DynamoDB suele ser lo suficientemente rápido (si no lo es, utilice DAX). Sin embargo, ElastiCache puede resultar más económico para los datos que se actualizan con poca frecuencia.

#### Optimización de costes
* No lea todo el elemento: Las unidades de capacidad de lectura utilizadas se basan en la cantidad de datos leídos. Utilice expresiones de proyección para definir qué atributos se recuperarán, de modo que sólo lea los datos que necesita.

* Filtre y ordene siempre basándose en la clave de ordenación: Puede filtrar y ordenar basándose en cualquier atributo. Si lo hace basándose en un atributo que es una clave de ordenación, DynamoDB utiliza el índice y usted solo paga por los elementos leídos. Si utiliza un atributo que no es una clave de ordenación, DynamoDB escanea toda la tabla y le cobra por cada elemento de la tabla. Esto es independiente de si consulta la clave de partición o no.

* No se exceda con los índices secundarios: Cada vez que escribe en una tabla, DynamoDB utiliza unidades de capacidad de escritura adicionales para actualizar los índices de esa tabla, lo que supone un coste adicional. Cree los índices que necesite, pero no más.

* Utilice la capacidad reservada: Puede reservar unidades de capacidad, igual que reservaría instancias en RDS.

* Prefiera el modo aprovisionado al modo bajo demanda: El modo bajo demanda es más sencillo, pero 5 veces más caro (sin capacidad reservada). El modo aprovisionado suele escalar lo suficientemente rápido, intente utilizarlo si su tráfico no aumenta tan rápido.

* Considere una tabla Standard-IA: Para la mayoría de las cargas de trabajo, una tabla estándar es la mejor opción. Pero para las cargas de trabajo que se leen con poca frecuencia, utilice la clase de tabla Standard-IA para reducir costes.

* Establecer un TTL: Algunos datos deben almacenarse para siempre, pero otros pueden eliminarse transcurrido cierto tiempo. Puedes automatizar esto estableciendo un TTL en cada elemento.

* No tenga miedo de utilizar varias bases de datos: DynamoDB es increíble para consultas sencillas con diferentes parámetros, y terrible para análisis complejos. No tenga miedo de utilizar una base de datos diferente para datos o casos de uso que no se ajusten a los puntos fuertes de DynamoDB.